## Imports and settings

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [2]:
pd.set_option('display.max_columns', None)

## Importing Data

In [3]:
df = pd.read_csv('../Data/AIDA_Results_IA_Institut.csv')

## Data exploration

In [4]:
df.head()

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event
0,1,Tasos Grillakis (GRC),M,FIM,NaN,00:00,33,23 m,YELLOW,12.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
1,2,Antonis Papantonatos (GRC),M,FIM,NaN,00:00,55,47 m,YELLOW,38.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
2,3,Dimitris Koumoulos (GRC),M,CNF,NaN,00:00,55,55 m,WHITE,55.0,-,Depth Event 2016,Depth Competition,2016-07-17,other
3,4,Christos Papadopoulos (GRC),M,CWT,NaN,00:00,55,55 m,WHITE,55.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other
4,5,Anna Chalari (GRC),F,CWT,NaN,00:00,15,15 m,WHITE,15.0,OK,Depth Event 2016,Depth Competition,2016-07-17,other


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26841 entries, 0 to 26840
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Start           26841 non-null  int64  
 1   Diver           26841 non-null  object 
 2   Gender          26841 non-null  object 
 3   Discipline      26841 non-null  object 
 4   Line            4694 non-null   float64
 5   Official Top    26841 non-null  object 
 6   AP              26841 non-null  int64  
 7   RP              26841 non-null  object 
 8   Card            26841 non-null  object 
 9   Points          26841 non-null  object 
 10  Remarks         26836 non-null  object 
 11  Title Event     26841 non-null  object 
 12  Event Type      26841 non-null  object 
 13  Day             26841 non-null  object 
 14  Category Event  26841 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 3.1+ MB


In [6]:
df.isna().sum()


Start                 0
Diver                 0
Gender                0
Discipline            0
Line              22147
Official Top          0
AP                    0
RP                    0
Card                  0
Points                0
Remarks               5
Title Event           0
Event Type            0
Day                   0
Category Event        0
dtype: int64

At this moment, we already know that we will drop columns "Start" and "Line", the first because it's basically a duplicated index, and the second because it only has 4694/26841 values, which make it useless.

On top of that, based on our needs, we know that we aren't interested in the columns "Official Top", "Title Event".

### Exploring Diver

In [7]:
df[df["Diver"].str.len() < 3]["Diver"]

2639     ()
4687     ()
5119     ()
5695     ()
5696     ()
5697     ()
7124     ()
7782     ()
7817     ()
7818     ()
7819     ()
8508     ()
8529     ()
8553     ()
8973     ()
8989     ()
9120     ()
9156     ()
9166     ()
10417    ()
10786    ()
10787    ()
10788    ()
11224    ()
13306    ()
13343    ()
Name: Diver, dtype: object

We have to note that some divers' name is missing, but we will handle this later in the modifcations part.

### Exploring Gender

In [8]:
df["Gender"].value_counts()

Gender
M    17434
F     9407
Name: count, dtype: int64

No problems here.

### Exploring Discipline

In [9]:
df["Discipline"].value_counts()

Discipline
CWT     10725
FIM      8075
CNF      4813
CWTB     3228
Name: count, dtype: int64

We don't have any problems here : there is only the values that we were expecting, with no NaN according to previous checks.

### Exploring AP

In [10]:
df["AP"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 26841 entries, 0 to 26840
Series name: AP
Non-Null Count  Dtype
--------------  -----
26841 non-null  int64
dtypes: int64(1)
memory usage: 209.8 KB


We only have numeric data so it's ok.

### Exploring RP

In [11]:
df["RP"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 26841 entries, 0 to 26840
Series name: RP
Non-Null Count  Dtype 
--------------  ----- 
26841 non-null  object
dtypes: object(1)
memory usage: 209.8+ KB


We observe that we don't only have numeric data, so we need to check manually.

In [12]:
for value in df["RP"].value_counts().index:
    print(value)

40 m
50 m
30 m
45 m
35 m
60 m
0 m
55 m
70 m
25 m
42 m
65 m
52 m
51 m
43 m
20 m
80 m
46 m
36 m
48 m
41 m
53 m
75 m
61 m
56 m
32 m
47 m
57 m
62 m
38 m
33 m
-2 m
58 m
63 m
31 m
37 m
54 m
66 m
90 m
28 m
44 m
71 m
72 m
85 m
67 m
68 m
73 m
64 m
34 m
27 m
26 m
15 m
82 m
49 m
78 m
81 m
76 m
22 m
77 m
74 m
-
59 m
23 m
88 m
83 m
100 m
91 m
39 m
92 m
86 m
21 m
95 m
29 m
24 m
84 m
69 m
18 m
93 m
87 m
10 m
17 m
96 m
16 m
101 m
105 m
94 m
12 m
98 m
19 m
89 m
102 m
103 m
97 m
79 m
14 m
8 m
9 m
106 m
110 m
13 m
108 m
11 m
5 m
104 m
60 m NR
99 m
107 m
40 m NR
7 m
111 m
50 m NR
3 m
30 m NR
114 m
6 m
65 m NR
112 m
75 m NR
71 m NR
45 m NR
116 m
80 m NR
52 m NR
120 m
109 m
55 m NR
4 m
117 m
61 m NR
118 m
72 m NR
81 m NR
70 m NR
123 m
100 m NR
113 m
115 m
57 m NR
1 m
35 m NR
90 m NR
2 m
76 m NR
66 m NR
74 m NR
20 m NR
25 m NR
119 m
88 m NR
121 m
53 m NR
62 m NR
106 m NR
101 m NR
83 m NR
54 m NR
42 m NR
46 m NR
85 m NR
125 m
77 m NR
51 m NR
67 m NR
36 m NR
41 m NR
73 m NR
92 m NR
82 m NR
102 m NR
97 m NR
122

In [13]:
len(df[df["RP"] == '-'])

138

We will only keep the int value, and replace '-' by 0 for the computation.

Moreover, 138 lines have no value specified as '-', which we replace by a 0.

In [14]:
df["RP"] = df['RP'].apply(lambda x: str(x).strip().split(' ')[0] if x != '-' else 0).astype(int)

In [15]:
df["RP"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 26841 entries, 0 to 26840
Series name: RP
Non-Null Count  Dtype
--------------  -----
26841 non-null  int32
dtypes: int32(1)
memory usage: 105.0 KB


We now only have int as values.

### Exploring Card

In [16]:
df["Card"].value_counts()

Card
WHITE     19189
YELLOW     4990
RED        2524
-           138
Name: count, dtype: int64

In [17]:
df[df["Card"] == "-"]

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event
25564,1,Chiara Obino (ITA),F,CWT,1.0,11:00,98,0,-,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-07,World Championship
25565,2,Daham Kim (KOR),M,CWTB,1.0,11:06,61,0,-,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-07,World Championship
25566,3,Rüstem Derİn (TUR),M,FIM,1.0,11:12,96,0,-,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-07,World Championship
25567,4,Mariam Shalan (EGY),F,FIM,1.0,11:18,60,0,-,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-07,World Championship
25568,5,Alejandro Lemus (MEX),M,CWT,1.0,11:24,92,0,-,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-07,World Championship
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25697,19,Jindřiška Zajacová (CZE),F,CWT,1.0,12:48,70,0,-,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-14,World Championship
25698,20,Lujain Talal (SAU),F,FIM,1.0,12:54,35,0,-,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-14,World Championship
25699,21,Arseniy Telegin (RUS),M,FIM,1.0,13:00,70,0,-,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-14,World Championship
25700,22,Essa Albarrk (SAU),M,FIM,1.0,13:06,35,0,-,-,-,AIDA Freediving World Cup August 2023,Depth Competition,2023-08-14,World Championship


In [18]:
df[df["Card"] == "-"]["RP"].value_counts()

RP
0    138
Name: count, dtype: int64

In [19]:
df[df["Card"] == "-"]["Remarks"].value_counts()

Remarks
-    138
Name: count, dtype: int64

Here, we clearly see that 138 lines are empty, so non interesting. We simply drop them.

In [20]:
df = df[df["Card"] != "-"]

### Exploring Points

In [21]:
df["Points"].info()

<class 'pandas.core.series.Series'>
Index: 26703 entries, 0 to 26840
Series name: Points
Non-Null Count  Dtype 
--------------  ----- 
26703 non-null  object
dtypes: object(1)
memory usage: 417.2+ KB


In [22]:
# Testing if we have non float values in the column
for value in df["Points"].value_counts().index:
    try:
        float(value)
    except:
        print(value)

In [23]:
df["Points"] = df["Points"].astype(float)

In [24]:
df["Points"].info()

<class 'pandas.core.series.Series'>
Index: 26703 entries, 0 to 26840
Series name: Points
Non-Null Count  Dtype  
--------------  -----  
26703 non-null  float64
dtypes: float64(1)
memory usage: 417.2 KB


Everything is now for sure normal here.

### Exploring Remarks

#### NaN corrections

We have seen that we have 5 NaN in this column.

Let's handle this first.

In [25]:
df[df["Remarks"].isna() == True]

,Start,Diver,Gender,Discipline,Line,Official Top,AP,RP,Card,Points,Remarks,Title Event,Event Type,Day,Category Event
122,5,Joan Capdevila (ESP),M,FIM,NaN,00:00,68,68,WHITE,68.0,NaN,2016 Yazbeck China Depth National Championship,Depth Competition,2016-08-15,other
186,1,Mara Torrealba Pascuet (ESP),F,FIM,NaN,00:00,55,37,YELLOW,18.0,NaN,Open Costa Brava,Depth Competition,2016-09-10,other
1075,3,Walid Boudhiaf (TUN),M,FIM,NaN,00:00,103,103,WHITE,103.0,NaN,Nirvana Oceanquest Competition,Depth Competition,2016-06-04,other
1138,1,Michel Filinis2 (BRA),M,FIM,NaN,00:00,45,45,WHITE,45.0,NaN,July Kalamata Qualifier Competition,Depth Competition,2016-07-11,other
1461,21,Paul Rollier (IRL),M,FIM,NaN,00:00,56,56,WHITE,56.0,NaN,Nice Abyss Contest 2016,Depth Competition,2016-06-18,NAC


In [26]:
df['Remarks'].fillna("-", inplace=True)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26703 entries, 0 to 26840
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Start           26703 non-null  int64  
 1   Diver           26703 non-null  object 
 2   Gender          26703 non-null  object 
 3   Discipline      26703 non-null  object 
 4   Line            4556 non-null   float64
 5   Official Top    26703 non-null  object 
 6   AP              26703 non-null  int64  
 7   RP              26703 non-null  int32  
 8   Card            26703 non-null  object 
 9   Points          26703 non-null  float64
 10  Remarks         26703 non-null  object 
 11  Title Event     26703 non-null  object 
 12  Event Type      26703 non-null  object 
 13  Day             26703 non-null  object 
 14  Category Event  26703 non-null  object 
dtypes: float64(2), int32(1), int64(2), object(10)
memory usage: 3.2+ MB


NaN have been replaced with a corresponding value.

#### Cleaning

This column is by far the most complex to clean, with a large number of unique values.

Hopefully, we have found patterns to facilitate the cleaning.

In [28]:
# In order to make modifications easier
df['Remarks'] = df['Remarks'].str.upper()

In [29]:
df['Remarks'].unique()

array(['-', 'OK', 'SHORT, TAG', '0', 'SHORT, NO TAG',
       'AP<RP, NO TAG, Y-ET', 'Y-BEFORE OT', 'RP AP, NO TAG, Y-ET',
       'DQ SP', 'GRAB', 'NR', 'DQ ??', 'DQPULL', 'DQBO-SURFACE', 'NO TAG',
       'NO TAG,UNDER AP', 'DQOTHER', 'DQ BO', 'DQSP', 'DNS',
       'NATIONAL RECORD FOR ESTONIA', 'NATIONAL RECORD FOR SPAIN',
       'SHORT NO TAG.', 'RED DQ AIRWAYS', 'NATIONAL RECORD FOR ESTONIA.',
       'NATIONAL RECORD FOR SPAIN.', 'NATIONAL RECORD FOR ISRAEL.',
       'RED DQSP', 'WORLD RECORD', 'NATIONAL RECORD',
       'EARLY TURN, NO TAG', 'BO', 'SP', 'GRAB X 2',
       'GRAB,NO TAG,UNDER AP', 'NO TAG,UNDER AP,DQPULL', 'DQBO', 'WR',
       'DQOTHER, PULLING', 'AP>RP NO TAG', 'DQ SP PULL',
       'SHORT TAG, EARLY TURN', 'DQSP ON NATIONAL RECORD',
       'DQOTHER, PULLING, NO TAG', 'CR', 'NATIONAL', 'CONTINENTAL RECORD',
       'SORT NO TAG', 'SHORT NO TAG', 'DQSP + SHORT, TAG',
       'DQSP (SEQUENCE)', 'DQSP (NOT PREFORMED), SHORT-TAG',
       'DQBO (AIRWAYS), SHORT-TAG', 'DQBO (A

In [30]:
def remarks_transformer(remark:str) -> str:
    if any(_ in remark for _ in ["DQBO", "DQBO-SURFACE", "DQBO-UW", "DQ BO", "BO", "BLACK OUT"]):
        return "Syncope"
    
    elif any(_ in remark for _ in ["R", "DQSP", "DQOTHER", "DNS", "DQAIRWAYS", "PULL", "DQ", "ASSIST", "SP", "LATE CHECKING", "DSN", "TOUCH", "COACH", "DIS", "DIP"]):
        return "RED"

    elif any(_ in remark for _ in ["Y", "PEN",  "NO TAG", "TAG", "AP", "RP", "EARLY", "TURN", "SHORT", "GRAB", "ET"]):
        return "YELLOW"

    elif any(_ in remark for _ in ["OK", "-", "NR", "RECORD", "W", "0", "DIVE TIME", "NATIONAL", "BIFINS", "PB", "78"]):
        return "WHITE"
    else:
        return remark

In [31]:
df["Remarks"] = df["Remarks"].apply(lambda x: remarks_transformer(str(x)))
df["Remarks"].value_counts()

Remarks
WHITE      18844
RED         4390
YELLOW      2696
Syncope      773
Name: count, dtype: int64

And we are good to go !

### Exploring Event Type

In [32]:
df["Event Type"].value_counts()

Event Type
Depth Competition      19297
Mixed Competition       3794
World Championship      1921
Competition             1447
Pool Competition         140
Worldrecord attempt       57
Team Competition          47
Name: count, dtype: int64

No missing values, and everything is as expected.

### Exploring Day

In [33]:
for value in df["Day"].value_counts().index:
    print(value)

2008-09-03
2012-11-20
2011-09-15
2013-09-15
2014-11-27
2014-05-25
2010-06-14
2010-09-26
2015-09-11
2015-05-23
2015-04-27
2020-08-15
2013-11-09
2008-07-12
2009-04-03
2023-06-24
2007-10-27
2012-09-09
2012-10-20
2009-04-21
2011-04-17
2003-05-29
2009-12-03
2006-12-09
2010-04-27
2008-04-18
2010-04-29
2013-05-20
2020-08-14
2015-09-05
2007-10-21
2022-06-18
2011-06-23
2015-08-27
2008-06-20
2009-09-06
2008-06-10
2013-09-09
2010-08-14
2002-10-31
2021-09-24
2017-08-26
2004-06-14
2007-08-14
2019-08-24
2021-09-25
2006-08-19
2016-04-30
2023-05-26
2006-09-09
2012-05-04
2015-06-06
2017-07-22
2019-06-09
2014-09-19
2005-09-01
2009-07-03
2015-11-01
2014-06-12
2023-03-10
2008-08-10
2013-10-05
2006-05-27
2022-08-06
2022-08-20
2015-06-20
2015-05-25
2014-06-15
2022-08-05
2016-05-02
2002-07-27
2009-07-19
2012-08-18
2022-09-11
2022-08-02
2008-07-25
2017-08-19
2016-05-01
2015-05-02
2021-09-17
2010-05-11
2021-06-19
2013-07-06
2009-06-22
2010-05-21
2009-08-13
2006-06-04
2013-10-30
2018-09-02
2019-09-21
2022-06-25

Data is consistent, so we may use it for further analysis.

### Exploring Category Event

In [34]:
df["Category Event"].value_counts()

Category Event
other                 18685
World Championship     4120
VB                     1880
Panglao                1339
NAC                     679
Name: count, dtype: int64

With that much divers in an unknown category, this isn't relevant to keep this column.

## Modifications and creation of specific columns

In [35]:
# Dropping duplicates if there are any (based on previous exploration, there isn't).
df.drop_duplicates(inplace=True)

### Dropping columns

In [36]:
df.drop(["Start", "Line", "Official Top", "Title Event", "Category Event"], axis=1, inplace=True)

In [37]:
df.head()

,Diver,Gender,Discipline,AP,RP,Card,Points,Remarks,Event Type,Day
0,Tasos Grillakis (GRC),M,FIM,33,23,YELLOW,12.0,WHITE,Depth Competition,2016-07-17
1,Antonis Papantonatos (GRC),M,FIM,55,47,YELLOW,38.0,WHITE,Depth Competition,2016-07-17
2,Dimitris Koumoulos (GRC),M,CNF,55,55,WHITE,55.0,WHITE,Depth Competition,2016-07-17
3,Christos Papadopoulos (GRC),M,CWT,55,55,WHITE,55.0,WHITE,Depth Competition,2016-07-17
4,Anna Chalari (GRC),F,CWT,15,15,WHITE,15.0,WHITE,Depth Competition,2016-07-17


### Creating new columns

#### Various columns

In [38]:
# Extracting the month from the 'Day' column
df['Month'] = pd.to_datetime(df['Day']).dt.month
df['Year'] = pd.to_datetime(df['Day']).dt.year

# Creating depth difference
df['Depth_Difference'] = df['AP'] - df['RP']

# Calculating total dive experience
# Sorting data by 'Day'
df.sort_values(by=['Day'], inplace=True)

# Cumulative count of dives per diver
df['Experience Dive'] = df.groupby('Diver').cumcount()

# Calculating experience per discipline
# Cumulative count of dives per diver per discipline
df['Experience Discipline'] = df.groupby(['Diver', 'Discipline']).cumcount()

In [39]:
# Removing "Day" column as it beacame useless
df.drop("Day", axis=1, inplace=True)

In [40]:
df.head()

,Diver,Gender,Discipline,AP,RP,Card,Points,Remarks,Event Type,Month,Year,Depth_Difference,Experience Dive,Experience Discipline
8929,Deborah Andollo (CUB),F,CWT,61,61,WHITE,61.0,WHITE,Worldrecord attempt,6,1994,0,0,0
3716,Umberto Pelizzari (ITA),M,CWT,72,72,WHITE,72.0,WHITE,Worldrecord attempt,9,1995,0,0,0
3713,Deborah Andollo (CUB),F,CWT,62,62,WHITE,62.0,WHITE,Worldrecord attempt,10,1996,0,1,1
5021,Michael Oliva (FRA),M,CWT,72,72,WHITE,72.0,WHITE,Worldrecord attempt,10,1996,0,0,0
3717,Alejandro Ravelo (CUB),M,CWT,73,73,WHITE,73.0,WHITE,Worldrecord attempt,8,1997,0,0,0


#### Creation of "Name" and "Country" columns from "Diver"

In [41]:
df[['Name', 'Country']] = df['Diver'].apply(lambda x:x.strip()).replace(" ()", " (Unknown)").replace("()", "Unknown (Unknown)").replace("", "Unknown (Unknown)").str.rsplit(' ', n=1, expand=True).astype(str).replace("()", "(Unknown)")

# Remove parentheses from 'Country' column values
df['Country'] = df['Country'].str.strip('()')

Thanks to that, we now have countries for future vizualisations.

Please note that unknown values as '()' or "Name Surname ()" have been marked as 'Unknown'.

In [42]:
df.head()

,Diver,Gender,Discipline,AP,RP,Card,Points,Remarks,Event Type,Month,Year,Depth_Difference,Experience Dive,Experience Discipline,Name,Country
8929,Deborah Andollo (CUB),F,CWT,61,61,WHITE,61.0,WHITE,Worldrecord attempt,6,1994,0,0,0,Deborah Andollo,CUB
3716,Umberto Pelizzari (ITA),M,CWT,72,72,WHITE,72.0,WHITE,Worldrecord attempt,9,1995,0,0,0,Umberto Pelizzari,ITA
3713,Deborah Andollo (CUB),F,CWT,62,62,WHITE,62.0,WHITE,Worldrecord attempt,10,1996,0,1,1,Deborah Andollo,CUB
5021,Michael Oliva (FRA),M,CWT,72,72,WHITE,72.0,WHITE,Worldrecord attempt,10,1996,0,0,0,Michael Oliva,FRA
3717,Alejandro Ravelo (CUB),M,CWT,73,73,WHITE,73.0,WHITE,Worldrecord attempt,8,1997,0,0,0,Alejandro Ravelo,CUB


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26700 entries, 8929 to 26783
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Diver                  26700 non-null  object 
 1   Gender                 26700 non-null  object 
 2   Discipline             26700 non-null  object 
 3   AP                     26700 non-null  int64  
 4   RP                     26700 non-null  int32  
 5   Card                   26700 non-null  object 
 6   Points                 26700 non-null  float64
 7   Remarks                26700 non-null  object 
 8   Event Type             26700 non-null  object 
 9   Month                  26700 non-null  int32  
 10  Year                   26700 non-null  int32  
 11  Depth_Difference       26700 non-null  int64  
 12  Experience Dive        26700 non-null  int64  
 13  Experience Discipline  26700 non-null  int64  
 14  Name                   26700 non-null  object 
 15  Coun

We can now drop "Diver" column as it became useless.

In [44]:
df.drop("Diver", axis=1, inplace=True)

### Reindexing DataFrame

In [45]:
df.columns

Index(['Gender', 'Discipline', 'AP', 'RP', 'Card', 'Points', 'Remarks',
       'Event Type', 'Month', 'Year', 'Depth_Difference', 'Experience Dive',
       'Experience Discipline', 'Name', 'Country'],
      dtype='object')

In [46]:
df = df.reindex(columns=['Name', 'Country', 'Gender', 'Discipline', 'AP', 'RP', 'Depth_Difference', 'Card', 'Points', 
'Remarks', 'Event Type', 'Month', 'Year', 'Experience Dive', 'Experience Discipline'])

In [47]:
df.head()

,Name,Country,Gender,Discipline,AP,RP,Depth_Difference,Card,Points,Remarks,Event Type,Month,Year,Experience Dive,Experience Discipline
8929,Deborah Andollo,CUB,F,CWT,61,61,0,WHITE,61.0,WHITE,Worldrecord attempt,6,1994,0,0
3716,Umberto Pelizzari,ITA,M,CWT,72,72,0,WHITE,72.0,WHITE,Worldrecord attempt,9,1995,0,0
3713,Deborah Andollo,CUB,F,CWT,62,62,0,WHITE,62.0,WHITE,Worldrecord attempt,10,1996,1,1
5021,Michael Oliva,FRA,M,CWT,72,72,0,WHITE,72.0,WHITE,Worldrecord attempt,10,1996,0,0
3717,Alejandro Ravelo,CUB,M,CWT,73,73,0,WHITE,73.0,WHITE,Worldrecord attempt,8,1997,0,0


### Exporting to csv

In [48]:
# Exporting data to a csv for other notebooks
df.to_csv("../Data/cleaned_df.csv")

## Transforming usefull categorical data to numeric data

In [49]:
# Reading cleaned csv, and droping useless columns for machine learning
df = pd.read_csv("../Data/cleaned_df.csv").drop(["Unnamed: 0", "Name", "Country"], axis=1)
df.head()

,Gender,Discipline,AP,RP,Depth_Difference,Card,Points,Remarks,Event Type,Month,Year,Experience Dive,Experience Discipline
0,F,CWT,61,61,0,WHITE,61.0,WHITE,Worldrecord attempt,6,1994,0,0
1,M,CWT,72,72,0,WHITE,72.0,WHITE,Worldrecord attempt,9,1995,0,0
2,F,CWT,62,62,0,WHITE,62.0,WHITE,Worldrecord attempt,10,1996,1,1
3,M,CWT,72,72,0,WHITE,72.0,WHITE,Worldrecord attempt,10,1996,0,0
4,M,CWT,73,73,0,WHITE,73.0,WHITE,Worldrecord attempt,8,1997,0,0


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26700 entries, 0 to 26699
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Gender                 26700 non-null  object 
 1   Discipline             26700 non-null  object 
 2   AP                     26700 non-null  int64  
 3   RP                     26700 non-null  int64  
 4   Depth_Difference       26700 non-null  int64  
 5   Card                   26700 non-null  object 
 6   Points                 26700 non-null  float64
 7   Remarks                26700 non-null  object 
 8   Event Type             26700 non-null  object 
 9   Month                  26700 non-null  int64  
 10  Year                   26700 non-null  int64  
 11  Experience Dive        26700 non-null  int64  
 12  Experience Discipline  26700 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 2.6+ MB


### Transforming

In [51]:
columns_to_transform = ["Gender", "Discipline", "Card", "Remarks", "Event Type"]

In [52]:
# Initializing one hot encoder
one_hot_encoder = OneHotEncoder()

In [53]:
# Initializing tranformer with desired settings
transformer = make_column_transformer(
    (OneHotEncoder(), columns_to_transform),
    remainder='passthrough')

In [54]:
# Transforming DataFrame's data
transformed = transformer.fit_transform(df)
transformed

array([[1.000e+00, 0.000e+00, 0.000e+00, ..., 1.994e+03, 0.000e+00,
        0.000e+00],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 1.995e+03, 0.000e+00,
        0.000e+00],
       [1.000e+00, 0.000e+00, 0.000e+00, ..., 1.996e+03, 1.000e+00,
        1.000e+00],
       ...,
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 2.023e+03, 1.000e+00,
        0.000e+00],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 2.023e+03, 1.000e+00,
        0.000e+00],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 2.023e+03, 2.000e+00,
        0.000e+00]])

### Converting to DataFrame

In [55]:
# Convert data to DataFrame
transformed_df = pd.DataFrame(
    transformed, 
    columns=transformer.get_feature_names_out()
)

In [56]:
# Checking data is complete
transformed_df.isna().sum()

onehotencoder__Gender_F                          0
onehotencoder__Gender_M                          0
onehotencoder__Discipline_CNF                    0
onehotencoder__Discipline_CWT                    0
onehotencoder__Discipline_CWTB                   0
onehotencoder__Discipline_FIM                    0
onehotencoder__Card_RED                          0
onehotencoder__Card_WHITE                        0
onehotencoder__Card_YELLOW                       0
onehotencoder__Remarks_RED                       0
onehotencoder__Remarks_Syncope                   0
onehotencoder__Remarks_WHITE                     0
onehotencoder__Remarks_YELLOW                    0
onehotencoder__Event Type_Competition            0
onehotencoder__Event Type_Depth Competition      0
onehotencoder__Event Type_Mixed Competition      0
onehotencoder__Event Type_Pool Competition       0
onehotencoder__Event Type_Team Competition       0
onehotencoder__Event Type_World Championship     0
onehotencoder__Event Type_World

In [57]:
transformed_df.head()

,onehotencoder__Gender_F,onehotencoder__Gender_M,onehotencoder__Discipline_CNF,onehotencoder__Discipline_CWT,onehotencoder__Discipline_CWTB,onehotencoder__Discipline_FIM,onehotencoder__Card_RED,onehotencoder__Card_WHITE,onehotencoder__Card_YELLOW,onehotencoder__Remarks_RED,onehotencoder__Remarks_Syncope,onehotencoder__Remarks_WHITE,onehotencoder__Remarks_YELLOW,onehotencoder__Event Type_Competition,onehotencoder__Event Type_Depth Competition,onehotencoder__Event Type_Mixed Competition,onehotencoder__Event Type_Pool Competition,onehotencoder__Event Type_Team Competition,onehotencoder__Event Type_World Championship,onehotencoder__Event Type_Worldrecord attempt,remainder__AP,remainder__RP,remainder__Depth_Difference,remainder__Points,remainder__Month,remainder__Year,remainder__Experience Dive,remainder__Experience Discipline
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,61.0,61.0,0.0,61.0,6.0,1994.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,72.0,72.0,0.0,72.0,9.0,1995.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,62.0,62.0,0.0,62.0,10.0,1996.0,1.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,72.0,72.0,0.0,72.0,10.0,1996.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,73.0,73.0,0.0,73.0,8.0,1997.0,0.0,0.0


### Renaming columns in a more friendly way

In [58]:
def renaming_columns(colunm_name):
    if colunm_name[:15] == 'onehotencoder__':
        return colunm_name[15:]
    elif colunm_name[:11] == 'remainder__':
        return colunm_name[11:]
    else:
        return colunm_name

In [59]:
transformed_df.rename(columns=renaming_columns, inplace=True)

In [60]:
transformed_df.columns

Index(['Gender_F', 'Gender_M', 'Discipline_CNF', 'Discipline_CWT',
       'Discipline_CWTB', 'Discipline_FIM', 'Card_RED', 'Card_WHITE',
       'Card_YELLOW', 'Remarks_RED', 'Remarks_Syncope', 'Remarks_WHITE',
       'Remarks_YELLOW', 'Event Type_Competition',
       'Event Type_Depth Competition', 'Event Type_Mixed Competition',
       'Event Type_Pool Competition', 'Event Type_Team Competition',
       'Event Type_World Championship', 'Event Type_Worldrecord attempt', 'AP',
       'RP', 'Depth_Difference', 'Points', 'Month', 'Year', 'Experience Dive',
       'Experience Discipline'],
      dtype='object')

### Exporting to .csv

In [61]:
# Exporting data to a csv for other notebooks
transformed_df.to_csv("../Data/for_ml_df.csv")